In [ ]:
# imports
import tensorflow as tf
import numpy as np
import seaborn as sns
import collections
from sklearn import metrics
 
import matplotlib.pyplot as plt
from typing import List

In [ ]:
# activation functions
def logistic(x, derivative=False):
    if derivative:
        return (np.exp(-x)) / ((np.exp(-x) + 1) ** 2)
    else:
        return np.exp(x) / (np.exp(x) + 1)


def leaky_relu(x, derivative=True):
    if derivative:
        dx = np.ones_like(x)
        dx[x < 0] = .01
        return dx
    else:
        return np.maximum(.01 * x, x)


def relu(x, derivative=False):
    if derivative:
        return (x > 0).astype(int)
    else:
        return np.maximum(0, x)


def softmax(x, derivative=False):
    # subtract the max of X to make softmax stable. as suggested by the top answer here :
    # https://stackoverflow.com/questions/61425412/stable-softmax-function-returns-wrong-output
    exps = np.exp(x - x.max())
    if derivative:
        return exps / np.sum(exps, axis=0) * (1. - exps / np.sum(exps, axis=0))
    else:
        return exps / np.sum(exps, axis=0)

def tanh(x, derivative=False):
  if derivative:
    return 1.0-(np.tanh(x)**2)
  else:
    return np.tanh(x)

In [ ]:
# vectorization function
def vectorize(x):
    # takes in  6000 [28x28] arrays. "flattens" them into 6000 [784x1] arrays 1 by 1.
    temp = []
    for i in range(x.shape[0]):
        temp.append(x[i].flatten())

    return np.array(temp)

In [ ]:
# load data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# normalize data
x_train, x_test = x_train[..., np.newaxis] / 255.0, x_test[..., np.newaxis] / 255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)
vectorized_x_train, vectorized_x_test = vectorize(x_train), vectorize(x_test)

In [ ]:
# Standardization

#mean_test = vectorized_x_test.mean().astype(np.float32)
#std_test = vectorized_x_test.std().astype(np.float32)
#vectorized_x_test = (vectorized_x_test - mean_test)/(std_test)

#mean_train = vectorized_x_train.mean().astype(np.float32)
#std_train = vectorized_x_train.std().astype(np.float32)
#vectorized_x_train = (vectorized_x_train - mean_train)/(std_train)

In [ ]:
class MultilayerPerceptron:

    def __init__(self, actFunction, nbOfLayers, nbOfUnits):

        # check that the input is valid
        assert len(nbOfUnits) == nbOfLayers
        self.actFunction = actFunction
        self.nbOfLayers = nbOfLayers
        self.nbOfUnits = nbOfUnits
        # temporarly initate lr to 0.01. will be overwritten with the desired Value in fit
        self.lr = 0.01
        # create a list that contains all layers and the number of neurons in them. the input and output layers are
        # hard coded for the MINST Digit data set. they would have been parameters if the Assigenment didn't limit
        # parameters to actFunction, nbOfLayers, nbOfUnits
        self.layers = [28 * 28] + nbOfUnits + [10]

        # initiate the weight and bias matrices with random values. np.random.randn gave me a better initial result
        # compared with rand. Uses Xavier initaliztion for weights http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf
        self.params = {}
        for i in range(len(self.layers) - 1):
            self.params[f'W{i + 1}'] = np.random.randn(self.layers[i + 1], self.layers[i]) * 1 / np.sqrt(self.layers[i])
            self.params[f'B{i + 1}'] = np.zeros(self.layers[i + 1])

    def forward_propogate(self, inputs):
        # activation for the input layer is just the input
        self.params['A0'] = inputs
        # compute the activation of each layer. Z is the net input and A is the activation
        for i in range(len(self.layers) - 1):
            self.params[f'Z{i + 1}'] = np.dot(self.params[f'W{i + 1}'], self.params[f'A{i}']) + self.params[f'B{i + 1}']
            if i != len(self.layers) - 1:
                self.params[f'A{i + 1}'] = self.actFunction(self.params[f'Z{i + 1}'])
            else:
                # last layer uses softmax
                self.params[f'A{i + 1}'] = softmax(self.params[f'Z{i + 1}'])

        # return the activation of the output layer(the "prediction").
        return self.params[f'A{len(self.layers) - 1}']

    def backwards_propagate(self, y_train, network_output, L2):
        N = network_output.shape
        # compute the error for the the output layer. divide by N to correct dimensions
        error = (network_output - y_train) * softmax(self.params[f'Z{len(self.layers) - 1}'], True) / N

        # traverse through the layers in reverse. from the last layer , [len - 1 ] , to the first hidden layer.
        # uses range ( start, end , step) function.

        for i in range(len(self.layers) - 1, 0, -1):
            # output, hidden , input
            # modify the weight based on the learning rate
            self.params[f'W{i}'] -= self.lr * ((np.outer(error, self.params[f'A{i - 1}'])) + L2 * self.params[f'W{i}'])
            self.params[f'B{i}'] -= self.lr * np.sum(error, axis=0)/N
            if i!=1:
                # self.params[W0] doesn't exist.so skip the i = 1 case. computes the error for the weight adjustment
                # of the next layer
                error = self.params[f'W{i}'].transpose().dot(error) * self.actFunction(self.params[f'Z{i - 1}'],
                                                                                       derivative=True)
        return self

    def fit(self, x, y, x_test, y_test, lr, iterations, variable_lr,L2 = 0.0, variable_duration=10, min_lr=0.001):
        # overwrite the default learning rate
        self.lr = lr
        predictions = []
        epochs = []
        training_accuracies = []
        testing_accuracies = []
        lr_counter = 0
        for i in range(iterations):
            # go through each example in the training set. record the predictions to compute the accuracy of the
            # training set
            for X, Y in zip(x, y):
                output = self.forward_propogate(X)
                prediction = np.argmax(output)
                label = np.argmax(Y)
                predictions.append(prediction == label)
                self.backwards_propagate(Y, output, L2)

            if variable_lr:
                self.lr = max(self.lr / (1 + int((lr_counter / variable_duration))), min_lr)
                lr_counter = lr_counter + 1
                if lr_counter > variable_duration:
                    lr_counter = 0
            

            # compute accuracy on the training and testing set to ensure learning and detect over-fitting and divergence
            epochs.append(i)
            a = self.accuracy(x_test, y_test)
            testing_accuracies.append(round(a * 100, 1))
            b = np.mean(predictions)
            training_accuracies.append(round(b * 100, 1))
           
            print(f'{i} iteration : test accuracy {round(a * 100, 1)}%, training accuracy  {round(b * 100, 1)}%. learning at {self.lr}')

        # Plotting testing and training accuracies     
        plt.plot(epochs, training_accuracies, 'g', label='Training Accuracy')
        plt.plot(epochs, testing_accuracies, 'b', label='Testing Accuracy')
        plt.title('Training and Testing accuracies vs Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

    def accuracy(self, x_test, y_test):
        predictions = []
        # compute accuracy by comparing the results of a forward propagate and the true label.
        # np.argmax was used to de-categorize
        for X, Y in zip(x_test, y_test):
            pre = self.predict(X)
            prediction = np.argmax(pre)
            label = np.argmax(Y)
            predictions.append(prediction == label)

        return np.array(predictions).mean()

    def getPredictions(self, x_test, y_test):
      # gets a list of the the predictions
      predictions = []
      for X, Y in zip(x_test, y_test):
            pre = self.predict(X)
            prediction = np.argmax(pre)
            label = np.argmax(Y)
            predictions.append(prediction)

      return np.array(predictions)


    def predict(self, x):
        return self.forward_propogate(x)

Expirement 1. Network Arch

In [ ]:
mlp = MultilayerPerceptron(relu, 2, [32,32])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron(relu, 2, [64,64])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

*Expirement* 2. # hidden layers


In [ ]:
mlp = MultilayerPerceptron(relu, 0, [])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron(relu, 1, [128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron(relu, 3, [128,128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

Expirement 3. Activation functions
NOTE: we found the optimal learning rate to be 0.9 for logistic and 0.01 for Relu/tanh 

In [ ]:
mlp = MultilayerPerceptron(tanh, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron(logistic, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.9, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

Expirement 4. Variable Learning Rate

Halves the learning rate every N itterations. set to 10 by default



In [ ]:
mlp = MultilayerPerceptron(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=250, variable_lr=True, variable_duration=10 )

Expirement 5. L2

In [ ]:
mlp = MultilayerPerceptron(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False, L2 = 0.0001)

Expirement 6. Xavier Initialization

With Xavier Initialization

In [ ]:
mlp = MultilayerPerceptron(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron(logistic, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.9, iterations=50, variable_lr=False)

Without Xavier Initilization(below is an MLP Class without Xavier Initilization)

In [ ]:
class MultilayerPerceptron2:

    def __init__(self, actFunction, nbOfLayers, nbOfUnits):

        # check that the input is valid
        assert len(nbOfUnits) == nbOfLayers
        self.actFunction = actFunction
        self.nbOfLayers = nbOfLayers
        self.nbOfUnits = nbOfUnits
        # temporarly initate lr to 0.01. will be overwritten with the desired Value in fit
        self.lr = 0.01
        # create a list that contains all layers and the number of neurons in them. the input and output layers are
        # hard coded for the MINST Digit data set. they would have been parameters if the Assigenment didn't limit
        # parameters to actFunction, nbOfLayers, nbOfUnits
        self.layers = [28 * 28] + nbOfUnits + [10]

        # initiate the weight and bias matrices with random values. np.random.randn gave me a better initial result
        # compared with rand. Uses Xavier initaliztion for weights http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf
        self.params = {}
        for i in range(len(self.layers) - 1):
            self.params[f'W{i + 1}'] = np.random.randn(self.layers[i + 1], self.layers[i])
            self.params[f'B{i + 1}'] = np.zeros(self.layers[i + 1])

    def forward_propogate(self, inputs):
        # activation for the input layer is just the input
        self.params['A0'] = inputs
        # compute the activation of each layer. Z is the net input and A is the activation
        for i in range(len(self.layers) - 1):
            self.params[f'Z{i + 1}'] = np.dot(self.params[f'W{i + 1}'], self.params[f'A{i}']) + self.params[f'B{i + 1}']
            if i != len(self.layers) - 1:
                self.params[f'A{i + 1}'] = self.actFunction(self.params[f'Z{i + 1}'])
            else:
                # last layer uses softmax
                self.params[f'A{i + 1}'] = softmax(self.params[f'Z{i + 1}'])

        # return the activation of the output layer(the "prediction").
        return self.params[f'A{len(self.layers) - 1}']

    def backwards_propagate(self, y_train, network_output, L2):
        N = network_output.shape
        # compute the error for the the output layer. divide by N to correct dimensions
        error = (network_output - y_train) * softmax(self.params[f'Z{len(self.layers) - 1}'], True) / N

        # traverse through the layers in reverse. from the last layer , [len - 1 ] , to the first hidden layer.
        # uses range ( start, end , step) function.

        for i in range(len(self.layers) - 1, 0, -1):
            # output, hidden , input
            # modify the weight based on the learning rate
            self.params[f'W{i}'] -= self.lr * ((np.outer(error, self.params[f'A{i - 1}'])) + L2 * self.params[f'W{i}'])
            self.params[f'B{i}'] -= self.lr * np.sum(error, axis=0)/N
            if i!=1:
                # self.params[W0] doesn't exist.so skip the i = 1 case. computes the error for the weight adjustment
                # of the next layer
                error = self.params[f'W{i}'].transpose().dot(error) * self.actFunction(self.params[f'Z{i - 1}'],
                                                                                       derivative=True)
        return self

    def fit(self, x, y, x_test, y_test, lr, iterations, variable_lr,L2 = 0.0, variable_duration=10, min_lr=0.001):
        # overwrite the default learning rate
        self.lr = lr
        predictions = []
        epochs = []
        training_accuracies = []
        testing_accuracies = []
        lr_counter = 0
        for i in range(iterations):
            # go through each example in the training set. record the predictions to compute the accuracy of the
            # training set
            for X, Y in zip(x, y):
                output = self.forward_propogate(X)
                prediction = np.argmax(output)
                label = np.argmax(Y)
                predictions.append(prediction == label)
                self.backwards_propagate(Y, output, L2)

            if variable_lr:
                self.lr = max(self.lr / (1 + int((lr_counter / variable_duration))), min_lr)
                lr_counter = lr_counter + 1
                if lr_counter > variable_duration:
                    lr_counter = 0
            

            # compute accuracy on the training and testing set to ensure learning and detect over-fitting and divergence
            epochs.append(i)
            a = self.accuracy(x_test, y_test)
            testing_accuracies.append(round(a * 100, 1))
            b = np.mean(predictions)
            training_accuracies.append(round(b * 100, 1))
           
            print(f'{i} iteration : test accuracy {round(a * 100, 1)}%, training accuracy  {round(b * 100, 1)}%. learning at {self.lr}')

        # Plotting testing and training accuracies     
        plt.plot(epochs, training_accuracies, 'g', label='Training Accuracy')
        plt.plot(epochs, testing_accuracies, 'b', label='Testing Accuracy')
        plt.title('Training and Testing accuracies vs Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

    def accuracy(self, x_test, y_test):
        predictions = []
        # compute accuracy by comparing the results of a forward propagate and the true label.
        # np.argmax was used to de-categorize
        for X, Y in zip(x_test, y_test):
            pre = self.predict(X)
            prediction = np.argmax(pre)
            label = np.argmax(Y)
            predictions.append(prediction == label)

        return np.array(predictions).mean()

    def getPredictions(self, x_test, y_test):
      # gets a list of the the predictions
      predictions = []
      for X, Y in zip(x_test, y_test):
            pre = self.predict(X)
            prediction = np.argmax(pre)
            label = np.argmax(Y)
            predictions.append(prediction)

      return np.array(predictions)


    def predict(self, x):
        return self.forward_propogate(x)

In [ ]:
mlp = MultilayerPerceptron2(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

In [ ]:
mlp = MultilayerPerceptron2(logistic, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.9, iterations=50, variable_lr=False)

Expirement 7. Normalization


In [ ]:
# Comment out standardization for this test

Expirement 8. Bias Term

With Bias Term

In [ ]:
mlp = MultilayerPerceptron(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)

Without Bias Term(Below is an MLP Class without the bias term being included in the activation or being updated)

In [ ]:
class MultilayerPerceptron3:

    def __init__(self, actFunction, nbOfLayers, nbOfUnits):

        # check that the input is valid
        assert len(nbOfUnits) == nbOfLayers
        self.actFunction = actFunction
        self.nbOfLayers = nbOfLayers
        self.nbOfUnits = nbOfUnits
        # temporarly initate lr to 0.01. will be overwritten with the desired Value in fit
        self.lr = 0.01
        # create a list that contains all layers and the number of neurons in them. the input and output layers are
        # hard coded for the MINST Digit data set. they would have been parameters if the Assigenment didn't limit
        # parameters to actFunction, nbOfLayers, nbOfUnits
        self.layers = [28 * 28] + nbOfUnits + [10]

        # initiate the weight and bias matrices with random values. np.random.randn gave me a better initial result
        # compared with rand
        self.params = {}
        for i in range(len(self.layers) - 1):
            self.params[f'W{i + 1}'] = np.random.randn(self.layers[i + 1], self.layers[i]) * 1 / np.sqrt(self.layers[i])
            self.params[f'B{i + 1}'] = np.zeros(self.layers[i + 1])

    def forward_propogate(self, inputs):
        # activation for the input layer is just the input
        self.params['A0'] = inputs
        # compute the activation of each layer. Z is the net input and A is the activation
        for i in range(len(self.layers) - 1):
            self.params[f'Z{i + 1}'] = np.dot(self.params[f'W{i + 1}'], self.params[f'A{i}']) 
            if i != len(self.layers) - 1:
                self.params[f'A{i + 1}'] = self.actFunction(self.params[f'Z{i + 1}'])
            else:
                # last layer uses softmax
                self.params[f'A{i + 1}'] = softmax(self.params[f'Z{i + 1}'])

        # return the activation of the output layer(the "prediction").
        return self.params[f'A{len(self.layers) - 1}']

    def backwards_propagate(self, y_train, network_output, L2):
        N = network_output.shape
        # compute the error for the the output layer. divide by N to correct dimensions
        error = (network_output - y_train) * softmax(self.params[f'Z{len(self.layers) - 1}'], True) / N

        # traverse through the layers in reverse. from the last layer , [len - 1 ] , to the first hidden layer.
        # uses range ( start, end , step) function.

        for i in range(len(self.layers) - 1, 0, -1):
            # output, hidden , input
            # modify the weight based on the learning rate
            self.params[f'W{i}'] -= self.lr * ((np.outer(error, self.params[f'A{i - 1}'])) + L2 * self.params[f'W{i}'])
            if i!=1:
                # self.params[W0] doesn't exist.so skip the i = 1 case. computes the error for the weight adjustment
                # of the next layer
                error = self.params[f'W{i}'].transpose().dot(error) * self.actFunction(self.params[f'Z{i - 1}'],
                                                                                       derivative=True)
        return self

    def fit(self, x, y, x_test, y_test, lr, iterations, variable_lr,L2 = 0.0, variable_duration=10, min_lr=0.001):
        # overwrite the default learning rate
        self.lr = lr
        predictions = []
        epochs = []
        training_accuracies = []
        testing_accuracies = []
        lr_counter = 0
        for i in range(iterations):
            # go through each example in the training set. record the predictions to compute the accuracy of the
            # training set
            for X, Y in zip(x, y):
                output = self.forward_propogate(X)
                prediction = np.argmax(output)
                label = np.argmax(Y)
                predictions.append(prediction == label)
                self.backwards_propagate(Y, output, L2)

            if variable_lr:
                self.lr = max(self.lr / (1 + int((lr_counter / variable_duration))), min_lr)
                lr_counter = lr_counter + 1
                if lr_counter > variable_duration:
                    lr_counter = 0
            

            # compute accuracy on the training and testing set to ensure learning and detect over-fitting and divergence
            epochs.append(i)
            a = self.accuracy(x_test, y_test)
            testing_accuracies.append(round(a * 100, 1))
            b = np.mean(predictions)
            training_accuracies.append(round(b * 100, 1))
           
            print(f'{i} iteration : test accuracy {round(a * 100, 1)}%, training accuracy  {round(b * 100, 1)}%. learning at {self.lr}')

        # Plotting testing and training accuracies     
        plt.plot(epochs, training_accuracies, 'g', label='Training Accuracy')
        plt.plot(epochs, testing_accuracies, 'b', label='Testing Accuracy')
        plt.title('Training and Testing accuracies vs Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

    def accuracy(self, x_test, y_test):
        predictions = []
        # compute accuracy by comparing the results of a forward propagate and the true label.
        # np.argmax was used to de-categorize
        for X, Y in zip(x_test, y_test):
            pre = self.predict(X)
            prediction = np.argmax(pre)
            label = np.argmax(Y)
            predictions.append(prediction == label)

        return np.array(predictions).mean()

    def getPredictions(self, x_test, y_test):
      # gets a list of the the predictions
      predictions = []
      for X, Y in zip(x_test, y_test):
            pre = self.predict(X)
            prediction = np.argmax(pre)
            label = np.argmax(Y)
            predictions.append(prediction)

      return np.array(predictions)


    def predict(self, x):
        return self.forward_propogate(x)

In [ ]:
mlp = MultilayerPerceptron3(relu, 2, [128,128])
mlp.fit(vectorized_x_train, y_train, vectorized_x_test, y_test, lr=0.01, iterations=50, variable_lr=False)